# Benchmarks

## Initialize

In [1]:
import os
import math
import pathlib
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from IPython.display import clear_output

import warnings
from lifelines.utils import CensoringType
from lifelines.utils import concordance_index

In [2]:
node = !hostname
if "sc" in node[0]:
    base_path = "/sc-projects/sc-proj-ukb-cvd"
else: 
    base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"
print(base_path)

project_label = "22_retina_phewas"
project_path = f"{base_path}/results/projects/{project_label}"
figure_path = f"{project_path}/figures"
output_path = f"{project_path}/data"

##### BEGIN ADAPT #####
# second best model
# wandb_name = 'aug++_convnext_s_mlp'
# wandb_id = '8ngm6apd'
# best model
wandb_name = 'aug++_convnext_s_mlp+'
wandb_id = '3p3smraz'
partitions = [0] # [i for i in range(22)]
##### END   ADAPT #####

experiment = wandb_id
experiment_path = f"{output_path}/{experiment}"
pathlib.Path(experiment_path).mkdir(parents=True, exist_ok=True)

/sc-projects/sc-proj-ukb-cvd


In [3]:
!ls {output_path}

3p3smraz			       eligable_eids_2022-05-24.feather
8ngm6apd			       eligable_eids_2022-05-25.feather
baseline_outcomes_220223.feather       eligable_eids_220511.feather
baseline_outcomes_220412	       eligable_eids_long_2022-05-24.feather
baseline_outcomes_220412.feather       eligable_eids_long_2022-05-25.feather
baseline_outcomes_long_220412.feather  eligable_eids_long_220511.feather
baseline_outcomes_wide_220301	       phecode_defs_220306.feather
baseline_outcomes_wide_220301.feather  retina_endpoints_220301.feather
baseline_outcomes_wide_220306.feather  test_experiment


In [4]:
endpoint_defs = pd.read_feather(f"{output_path}/phecode_defs_220306.feather").sort_values("endpoint")

In [5]:
data_outcomes = pd.read_feather(f"{output_path}/baseline_outcomes_220412.feather").set_index("eid")

In [6]:
from datetime import date
today = str(date.today())

In [7]:
eligable_eids = pd.read_feather(f"{output_path}/eligable_eids_{today}.feather")
eids_dict = eligable_eids.set_index("endpoint")["eid_list"].to_dict()

In [8]:
import pandas as pd
endpoints = sorted([l.replace('_prevalent', '') for l in list(pd.read_csv('/sc-projects/sc-proj-ukb-cvd/results/projects/22_medical_records/data/220413/frequent_endpoints.csv').endpoint.values)])

In [9]:
#endpoints_in_eligable_eids_unique = eligable_eids['endpoint'].unique()
#endpoints_in_eligable_eids = [e for e in endpoints if e in endpoints_in_eligable_eids_unique]

d = []
for endpoint in tqdm(endpoints): # TEST: in tqdm(endpoints_in_eligable_eids): # TODO CHANGE!
    n = data_outcomes[f"{endpoint}_event"].loc[eids_dict[endpoint]].sum()
    freq = data_outcomes[f"{endpoint}_event"].loc[eids_dict[endpoint]].mean()
    d.append({"endpoint": endpoint, "n": n, "freq": freq})

  0%|          | 0/498 [00:00<?, ?it/s]

In [10]:
endpoints_freqs = pd.DataFrame().from_dict(d)

In [11]:
endpoints_ds = endpoints_freqs.query("n>100").sort_values("endpoint").reset_index(drop=True)

In [12]:
endpoints_ds.to_csv(f"{experiment_path}/endpoints.csv")

In [13]:
f"{experiment_path}/endpoints.csv"

'/sc-projects/sc-proj-ukb-cvd/results/projects/22_retina_phewas/data/3p3smraz/endpoints.csv'